In [512]:
import os
import sys
import torch
from datasets import load_dataset, DatasetDict,Dataset,Audio
from tqdm import tqdm
from transformers.pipelines.pt_utils import     KeyDataset
from transformers import (
    AutoFeatureExtractor,
    WhisperTokenizer,
    AutoModelForSpeechSeq2Seq,
    AutoModelForSeq2SeqLM,
    AutoProcessor,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    pipeline,
    AutoTokenizer,
    TrainerCallback, 
    TrainingArguments, 
    TrainerState, 
    TrainerControl
)
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
from torch.utils.data import DataLoader
from dataclasses import dataclass
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import evaluate
from evaluate import load
import pandas as pd
import os
from sklearn.model_selection import train_test_split

torch.set_num_threads(1)

In [513]:
#csv = pd.read_csv("/root/dataset/swa_manifest.csv",encoding='utf8', engine="python")
csv = pd.read_csv("/root/ccai/tmp/youtube_dataset/dataset_3/m.csv",encoding='utf8', engine="python")
#csv = pd.read_csv("/root/ccai/tmp/youtube_dataset/dataset (4)/manifest.csv", engine="python")
csv = csv.rename(columns={"transcript":"sentence"})

In [514]:
csv.shape

(1768, 3)

In [515]:
csv.isnull().sum()

file_name       0
path         1301
sentence     1003
dtype: int64

In [516]:
# Will use below for whole dataset

#csv['path'] = "/root/dataset/swanglish"+csv['file_name']

In [517]:
csv['path'] = "/root/ccai/tmp/youtube_dataset/dataset_3/English/"+csv['file_name']
#csv = csv.dropna()

In [518]:
csv['path']

0       /root/ccai/tmp/youtube_dataset/dataset_3/Engli...
1       /root/ccai/tmp/youtube_dataset/dataset_3/Engli...
2       /root/ccai/tmp/youtube_dataset/dataset_3/Engli...
3       /root/ccai/tmp/youtube_dataset/dataset_3/Engli...
4       /root/ccai/tmp/youtube_dataset/dataset_3/Engli...
                              ...                        
1763    /root/ccai/tmp/youtube_dataset/dataset_3/Engli...
1764    /root/ccai/tmp/youtube_dataset/dataset_3/Engli...
1765    /root/ccai/tmp/youtube_dataset/dataset_3/Engli...
1766    /root/ccai/tmp/youtube_dataset/dataset_3/Engli...
1767    /root/ccai/tmp/youtube_dataset/dataset_3/Engli...
Name: path, Length: 1768, dtype: object

In [519]:
csv['sentence'] = csv['sentence'].fillna('NULL')

In [520]:
csv['sentence'] = csv['sentence'].astype(str)

In [521]:
csv

,file_name,path,sentence
0,11391369_i-knew-the-friendship-was-over-when-e...,/root/ccai/tmp/youtube_dataset/dataset_3/Engli...,be honest if one of your friends not me obviou...
1,11391369_i-knew-the-friendship-was-over-when-e...,/root/ccai/tmp/youtube_dataset/dataset_3/Engli...,our energy is back yeah because now we're like...
2,11391369_i-knew-the-friendship-was-over-when-e...,/root/ccai/tmp/youtube_dataset/dataset_3/Engli...,tmi wednesday at 12 you must get an episode wh...
3,11391369_i-knew-the-friendship-was-over-when-e...,/root/ccai/tmp/youtube_dataset/dataset_3/Engli...,handles here we talk about honest conversation...
4,11391369_i-knew-the-friendship-was-over-when-e...,/root/ccai/tmp/youtube_dataset/dataset_3/Engli...,um we don't know what to say um as we say we b...
...,...,...,...
1763,35795425_full-budget-reading-for-the-finacial-...,/root/ccai/tmp/youtube_dataset/dataset_3/Engli...,NULL
1764,35795425_full-budget-reading-for-the-finacial-...,/root/ccai/tmp/youtube_dataset/dataset_3/Engli...,designated based on two strategic pillars.. T...
1765,35795425_full-budget-reading-for-the-finacial-...,/root/ccai/tmp/youtube_dataset/dataset_3/Engli...,NULL
1766,35795425_full-budget-reading-for-the-finacial-...,/root/ccai/tmp/youtube_dataset/dataset_3/Engli...,"agro-industry in citrus fruits, vegetable oil,..."


In [522]:
data = DatasetDict()

## Below will be used for the original dataset

# data= load_dataset(
#     "audiofolder",
#     data_dir="/root/dataset/swanglish",
#     split="train",
#     trust_remote_code=True
# )

data= load_dataset(
    "audiofolder",
    data_dir="/root/ccai/tmp/youtube_dataset/dataset_3/English",
    split="train",
    trust_remote_code=True
)

In [523]:
data

Dataset({
    features: ['audio'],
    num_rows: 673
})

In [524]:
# # Perform a train-test split on the 'train' subset of the dataset
# train_test_split = data.train_test_split(test_size=0.2, seed=42)

# # Now, further split the training set into training and validation sets
# train_validation_split = train_test_split['train'].train_test_split(test_size=0.25, seed=42)  # 0.25 x 0.8 = 0.2

# # Create a new DatasetDict to include the split datasets

# split_dataset = DatasetDict({
#     'train': train_validation_split['train'],
#     'validation': train_validation_split['test'],
#     'test': train_test_split['test']
# })

# or_split_dataset = split_dataset
# print(split_dataset)

In [525]:
# Load a dataset into a DatasetDict

# Perform a train-test split on the 'train' subset of the dataset
train_test_split = data.train_test_split(test_size=0.2)

# Create a new DatasetDict to include the split datasets
split_dataset = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test'],
    #'validation': dataset['validation']  # Assuming the original dataset has a 'validation' set
})

or_split_dataset = split_dataset
print(split_dataset)

DatasetDict({
    train: Dataset({
        features: ['audio'],
        num_rows: 538
    })
    test: Dataset({
        features: ['audio'],
        num_rows: 135
    })
})


In [526]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['audio'],
        num_rows: 538
    })
    test: Dataset({
        features: ['audio'],
        num_rows: 135
    })
})

In [527]:
csv.shape

(1768, 3)

In [528]:
csv.columns

Index(['file_name', 'path', 'sentence'], dtype='object')

In [529]:
new_feature_dict = csv.set_index('path')['sentence'].to_dict()

In [530]:
new_feature_dict

{'/root/ccai/tmp/youtube_dataset/dataset_3/English/11391369_i-knew-the-friendship-was-over-when-ep-95_seg0.mp3': "be honest if one of your friends not me obviously i'm top tier yeah i'm slept with your boyfriend.  your husband would you end them both hi guys maruki muni here it's lady akm and we're back.  again with another episode of the messy in between it's definitely tmi",
 '/root/ccai/tmp/youtube_dataset/dataset_3/English/11391369_i-knew-the-friendship-was-over-when-ep-95_seg1.mp3': "our energy is back yeah because now we're like this is the last episode we're shooting for a.  while like three i'm like yes let's give it the best we got yeah and then go sleep don't panic.  episodes are still coming out it's just a production break yeah that is all by the way.  one thing about us is that we are more concerned about consistency and commitment to tmi than we.  are to anything else in life including our own pages including our marriages that one can take.  a pause but tmi",
 '/root/cca

In [531]:
split_dataset['train']

Dataset({
    features: ['audio'],
    num_rows: 538
})

In [532]:
split_dataset['train'][2]

{'audio': {'path': '/root/ccai/tmp/youtube_dataset/dataset_3/English/11391369_i-knew-the-friendship-was-over-when-ep-95_seg71.mp3',
  'array': array([-0.01729894, -0.01761993, -0.01555785, ..., -0.01672047,
         -0.01200814, -0.0061167 ]),
  'sampling_rate': 16000}}

In [533]:
def add_new_feature(batch):
    
    batch['sentence'] = new_feature_dict.get(batch['audio']['path'], 'NULL')  # Use None if the text is not found
    #print('Returned = ', batch)
    return batch

In [534]:
add_new_feature(split_dataset['train'][5])

{'audio': {'path': '/root/ccai/tmp/youtube_dataset/dataset_3/English/24806782_understanding-attachment-styles-and-dating-with-an-anxious-attachment-style-ft-lydia-km-bksp-ep-2_seg88.mp3',
  'array': array([-0.01236621, -0.01058473, -0.00048488, ..., -0.00052596,
         -0.00076587, -0.00084009]),
  'sampling_rate': 16000},
 'sentence': 'NULL'}

In [535]:
split_dataset['train'].features

{'audio': Audio(sampling_rate=None, mono=True, decode=True, id=None)}

In [536]:
split_dataset = split_dataset.map(add_new_feature, num_proc=3)

Map (num_proc=3): 100%|██████████| 135/135 [00:20<00:00,  6.75 examples/s]


In [537]:
or_split_dataset = or_split_dataset.map(add_new_feature, num_proc=3)

In [538]:
for data in split_dataset['train']:
    print(data)

{'audio': {'path': None, 'array': array([ 0.00817871,  0.00805664,  0.00601196, ..., -0.05227661,
       -0.02877808,  0.00436401]), 'sampling_rate': 16000}, 'sentence': "I don't know how I would put a figure now, but I think I would just put it in terms of time..  I would say a good savings really is just six months, assuming you can work very hard and get something within six months..  Now, everyone's livelihood and cash flow will determine what the six-month amount would look like..  But I would say about six months' worth of whatever you obligate."}
{'audio': {'path': None, 'array': array([0.0489502 , 0.0798645 , 0.12081909, ..., 0.03018188, 0.02734375,
       0.02365112]), 'sampling_rate': 16000}, 'sentence': 'NULL'}
{'audio': {'path': None, 'array': array([-0.01730347, -0.01763916, -0.01556396, ..., -0.01672363,
       -0.01202393, -0.00613403]), 'sampling_rate': 16000}, 'sentence': 'NULL'}
{'audio': {'path': None, 'array': array([-0.00366211, -0.00366211, -0.00384521, ..., -0.00

{'audio': {'path': None, 'array': array([-0.00949097, -0.00860596, -0.00604248, ..., -0.02850342,
       -0.021698  , -0.01123047]), 'sampling_rate': 16000}, 'sentence': "and then you're just yeah for me i think i mean i've dated a bunch of avoidance for sure as well.  i mean not a bunch but i have dated avoidance but i think i have dated one that was like that.  that he was very uncomfortable with emotions and when it did end it was very no there was no like.  cheating or anything it was just like yeah okay no this is not like working out anymore i don't.  feel like my emotional needs are being met and and all of that"}
{'audio': {'path': None, 'array': array([-0.0083313 , -0.00942993, -0.00891113, ..., -0.03973389,
       -0.03601074, -0.03091431]), 'sampling_rate': 16000}, 'sentence': 'NULL'}
{'audio': {'path': None, 'array': array([-0.00283813,  0.00424194, -0.00012207, ..., -0.00250244,
        0.00125122, -0.00247192]), 'sampling_rate': 16000}, 'sentence': 'of the year 2022, as w

In [539]:
def filter_none_features(batch): 
    if batch['sentence'] != "NULL":
        return batch
    else:
        return None

In [540]:
for key in split_dataset.keys():
    split_dataset[key] = split_dataset[key].filter(filter_none_features)

Filter: 100%|██████████| 135/135 [00:00<00:00, 382.10 examples/s]


In [541]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 221
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 44
    })
})

In [542]:
for record in split_dataset['train']:
    print(record)

{'audio': {'path': None, 'array': array([ 0.00817871,  0.00805664,  0.00601196, ..., -0.05227661,
       -0.02877808,  0.00436401]), 'sampling_rate': 16000}, 'sentence': "I don't know how I would put a figure now, but I think I would just put it in terms of time..  I would say a good savings really is just six months, assuming you can work very hard and get something within six months..  Now, everyone's livelihood and cash flow will determine what the six-month amount would look like..  But I would say about six months' worth of whatever you obligate."}
{'audio': {'path': None, 'array': array([-0.00216675, -0.00222778, -0.00167847, ...,  0.06832886,
        0.06192017,  0.05044556]), 'sampling_rate': 16000}, 'sentence': "Oh, really?.  Yes, because we have assigned parking..  So everyone will know whose it is..  And I was just going through that process of, okay, what are they going to think?.  There's the expectation, someone at my level, SUVs, the big luxury cars, Mercedes, BMW, their

{'audio': {'path': None, 'array': array([-0.01065063,  0.02432251,  0.05456543, ...,  0.00811768,
        0.00769043,  0.00787354]), 'sampling_rate': 16000}, 'sentence': "How is it, Toka?.  I don't..  Won't go..  Call me something..  That's me..  You call me dark chocolate..  That's me, bro..  I like it..  Seriously..  I said I don't..  What?.  Like, I said I don't..  And your skin actually looks like this..  Everyone look here..  Isn't that true?.  Maybe you have oily skin?.  I think so..  But I don't apply it..  Okay..  I hate doing that because I don't like it."}
{'audio': {'path': None, 'array': array([-0.00701904, -0.0055542 , -0.00888062, ..., -0.00933838,
       -0.00506592,  0.00997925]), 'sampling_rate': 16000}, 'sentence': "um we don't know what to say um as we say we bring the consistency we bring the content you bring the.  what subscription support that enables us to keep making videos about things that we love things.  that you guys love and things i want to keep talking 

In [543]:
# for key in split_dataset.keys():
#     split_dataset[key] = split_dataset[key].filter(filter_none_features)

In [544]:
for key in split_dataset.keys():
    or_split_dataset[key] = or_split_dataset[key].filter(filter_none_features)

In [545]:
split_dataset['train'].features

{'audio': Audio(sampling_rate=None, mono=True, decode=True, id=None),
 'sentence': Value(dtype='string', id=None)}

In [546]:
for record in split_dataset['train']:
    print(record)

{'audio': {'path': None, 'array': array([ 0.00817871,  0.00805664,  0.00601196, ..., -0.05227661,
       -0.02877808,  0.00436401]), 'sampling_rate': 16000}, 'sentence': "I don't know how I would put a figure now, but I think I would just put it in terms of time..  I would say a good savings really is just six months, assuming you can work very hard and get something within six months..  Now, everyone's livelihood and cash flow will determine what the six-month amount would look like..  But I would say about six months' worth of whatever you obligate."}
{'audio': {'path': None, 'array': array([-0.00216675, -0.00222778, -0.00167847, ...,  0.06832886,
        0.06192017,  0.05044556]), 'sampling_rate': 16000}, 'sentence': "Oh, really?.  Yes, because we have assigned parking..  So everyone will know whose it is..  And I was just going through that process of, okay, what are they going to think?.  There's the expectation, someone at my level, SUVs, the big luxury cars, Mercedes, BMW, their

{'audio': {'path': None, 'array': array([-0.00967407, -0.00311279,  0.00244141, ...,  0.00149536,
        0.00286865,  0.00079346]), 'sampling_rate': 16000}, 'sentence': 'Omoona, mazingila haya, omoona, kukinachofanyika hapa, tunaenda, moja kwa moja, ehe, omoona, sasa, mazingila haya, ehe, mazingila haya, ehe, mazingila haya, omoona, kinachofanyika sasa hivye, nimewaona wale, ehe, kinachofanyika, ehe, sasa, omoona, kamilamani swagia kidogo, ehe, omoona, kuna odada nimewaona pale mbele, omoona, wale pale, sija,'}
{'audio': {'path': None, 'array': array([-0.06460571, -0.08071899, -0.07885742, ..., -0.16033936,
       -0.16055298, -0.13708496]), 'sampling_rate': 16000}, 'sentence': "below it we all go through the same problems.  we all go through the same struggles.  I'm not saying.  mine are more than yours.  but we all go through the.  same problems.  and the same turmoil in terms.  of self doubt and trying.  to dig yourself out.  of the problem.  so.  don't judge others.  it's easy.  w

In [547]:
large_model_name = "openai/whisper-small"  # Example model name, use the correct one

## Create the Feature extractor
feature_extractor = AutoFeatureExtractor.from_pretrained(large_model_name)


# ## create the tokenizer
# tokenizer = WhisperTokenizer.from_pretrained(
#     large_model_name, language="sw", task="transcribe",max_length=249
# )


tokenizer = WhisperTokenizer.from_pretrained(large_model_name, language ='swahili' ,trust_remote_code=True, device_map="auto",padding=True, truncation=True)
#tokenizer.pad_token = tokenizer.eos_token
#tokenizer.padding_side = "right"


## Whisper processor
processor = AutoProcessor.from_pretrained(
    large_model_name,
    language="swahili", 
    task="transcribe"
)

/root/fine-tuning/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [548]:
input_str = split_dataset["train"][1]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

In [549]:
print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")
print(f"Labels :                        {labels}")

Input:                 Oh, really?.  Yes, because we have assigned parking..  So everyone will know whose it is..  And I was just going through that process of, okay, what are they going to think?.  There's the expectation, someone at my level, SUVs, the big luxury cars, Mercedes, BMW, their mental expectations people have..  So I had to go through that process and just say, you know what?.  I'm living my life..  These are things I like.
Decoded w/ special:    <|startoftranscript|><|sw|><|notimestamps|>Oh, really?.  Yes, because we have assigned parking..  So everyone will know whose it is..  And I was just going through that process of, okay, what are they going to think?.  There's the expectation, someone at my level, SUVs, the big luxury cars, Mercedes, BMW, their mental expectations people have..  So I had to go through that process and just say, you know what?.  I'm living my life..  These are things I like.<|endoftext|>
Decoded w/out special: Oh, really?.  Yes, because we have as

In [550]:
feature_extractor

WhisperFeatureExtractor {
  "chunk_length": 30,
  "feature_extractor_type": "WhisperFeatureExtractor",
  "feature_size": 80,
  "hop_length": 160,
  "n_fft": 400,
  "n_samples": 480000,
  "nb_max_frames": 3000,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "WhisperProcessor",
  "return_attention_mask": false,
  "sampling_rate": 16000
}

In [551]:
tokenizer

WhisperTokenizer(name_or_path='openai/whisper-small', vocab_size=50258, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|endoftext|>', '<|startoftranscript|>', '<|en|>', '<|zh|>', '<|de|>', '<|es|>', '<|ru|>', '<|ko|>', '<|fr|>', '<|ja|>', '<|pt|>', '<|tr|>', '<|pl|>', '<|ca|>', '<|nl|>', '<|ar|>', '<|sv|>', '<|it|>', '<|id|>', '<|hi|>', '<|fi|>', '<|vi|>', '<|he|>', '<|uk|>', '<|el|>', '<|ms|>', '<|cs|>', '<|ro|>', '<|da|>', '<|hu|>', '<|ta|>', '<|no|>', '<|th|>', '<|ur|>', '<|hr|>', '<|bg|>', '<|lt|>', '<|la|>', '<|mi|>', '<|ml|>', '<|cy|>', '<|sk|>', '<|te|>', '<|fa|>', '<|lv|>', '<|bn|>', '<|sr|>', '<|az|>', '<|sl|>', '<|kn|>', '<|et|>', '<|mk|>', '<|br|>', '<|eu|>', '<|is|>', '<|hy|>', '<|ne|>', '<|mn|>', '<|bs|>', '<|kk|>', '<|sq|>', '<|sw|>', '<|gl|>', '<|mr|>', '<|pa|>', '<|si

In [552]:
def data_preprocessing(batch):
    # load and resample audio data from 48 to 16kHz
    #print('batch = ',batch)
    #print('batch = ',batch)
    audio = batch["audio"]
    #print('audio = ',audio)

    # create Features that will be fed into the model
    batch["input_features"] = feature_extractor(
        audio["array"], sampling_rate=audio["sampling_rate"]
    ).input_features[0]

    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    
    #print('Output = ', batch)
    
    return batch

In [553]:
data_preprocessing(split_dataset["train"][1])

{'audio': {'path': None,
  'array': array([-0.00216675, -0.00222778, -0.00167847, ...,  0.06832886,
          0.06192017,  0.05044556]),
  'sampling_rate': 16000},
 'sentence': "Oh, really?.  Yes, because we have assigned parking..  So everyone will know whose it is..  And I was just going through that process of, okay, what are they going to think?.  There's the expectation, someone at my level, SUVs, the big luxury cars, Mercedes, BMW, their mental expectations people have..  So I had to go through that process and just say, you know what?.  I'm living my life..  These are things I like.",
 'input_features': array([[-0.02484369, -0.20047069, -0.36883843, ...,  0.20580977,
          0.21298033, -0.09294295],
        [-0.2094872 ,  0.013785  , -0.17807233, ...,  0.74393976,
          0.7734855 ,  0.7823543 ],
        [-0.14482749,  0.03909487,  0.02860826, ...,  0.8037658 ,
          0.85864186,  0.9056647 ],
        ...,
        [-0.62212205, -0.62212205, -0.62212205, ..., -0.26205122

In [554]:
split_dataset = split_dataset.map(data_preprocessing, remove_columns=["audio", "sentence"], num_proc=3)

Map (num_proc=3): 100%|██████████| 44/44 [00:00<00:00, 57.27 examples/s]


In [555]:
split_dataset['train']

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 221
})

In [557]:
split_dataset['train'].features

{'input_features': Sequence(feature=Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [558]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [559]:
model = AutoModelForSpeechSeq2Seq.from_pretrained(large_model_name, use_cache=False).to(
        device
    )
    
model.generation_config.language = 'swahili'
model.generation_config.task ='transcribe'
    
model.generation_config.forced_decoder_ids = None
    
    
print(model)

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
        

In [560]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor ([`WhisperProcessor`])
            The processor used for processing the data.
        decoder_start_token_id (`int`)
            The begin-of-sentence of the decoder.
        forward_attention_mask (`bool`)
            Whether to return attention_mask.
    """

    processor: Any
    decoder_start_token_id: int
#     forward_attention_mask: bool

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        model_input_name = self.processor.model_input_names[0]
        input_features = [
            {model_input_name: feature[model_input_name]} for feature in features
        ]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.feature_extractor.pad(
            input_features, return_tensors="pt"
        )


        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        
        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]
        
        # replace padding with -100 to ignore loss correctly
#         labels = labels_batch["input_ids"].masked_fill(
#             labels_batch.attention_mask.ne(1), -100
#         )

#         # if bos token is appended in previous tokenization step,
#         # cut bos token here as it's append later anyways
#         if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
#             labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [561]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id = model.config.decoder_start_token_id
)

In [562]:
# training_args = Seq2SeqTrainingArguments(
#     output_dir="./whisper-small-hi", # Change to a repo name of your choice
#     per_device_train_batch_size=16, # Adjust based on your GPU memory
#     gradient_accumulation_steps=1, # Increase by 2x for every 2x decrease in batch size
#     learning_rate=1e-5, # Start with a lower learning rate for fine-tuning
#     #warmup_steps=500, # Adjust based on your training schedule
#     max_steps=5000, # Increase for more training steps
#     num_train_epochs=3, # Specify the number of epochs for training
#     gradient_checkpointing=True, # Use gradient checkpointing for memory efficiency
#     fp16=True, # Use mixed precision training for faster training
#     evaluation_strategy="steps", # Evaluate model at regular intervals
#     per_device_eval_batch_size=8, # Adjust based on your GPU memory
#     predict_with_generate=True, # Use generate method for prediction
#     generation_max_length=225, # Set the maximum length for generated sequences
#     save_steps=1000, # Save model checkpoints every 1000 steps
#     eval_steps=1000, # Evaluate model every 1000 steps
#     logging_steps=25, # Log training progress every 25 steps
#     report_to=["tensorboard"], # Use TensorBoard for logging
#     load_best_model_at_end=True, # Load the best model at the end of training
#     metric_for_best_model="wer", # Use Word Error Rate (WER) as the metric for best model
#     greater_is_better=False, # Since lower WER is better
#     #push_to_hub=True, # Push model checkpoints to Hugging Face Hub
# )

# training_args = Seq2SeqTrainingArguments(
#     output_dir="./fine-tuned-model",
#     per_device_train_batch_size=8,
#     gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
#     learning_rate=1e-5,
#     warmup_steps=50,
#     #max_steps=5000,
#     #auto_find_batch_size = True,
#     num_train_epochs=200,  # Adjust this based on your desired number of epochs
#     # gradient_checkpointing=True,
#     #remove_unused_columns=False,
#     report_to=["tensorboard"], # Use TensorBoard for logging
#     load_best_model_at_end=True, # Load the best model at the end of training
#     fp16=True,
#     evaluation_strategy="steps",
#     per_device_eval_batch_size=8,
#     predict_with_generate=True,
#     generation_max_length=250,
#     save_steps =500,
#     label_names=["labels"],
#     greater_is_better=True
#     #push_to_hub=True,
# )
training_args = Seq2SeqTrainingArguments(
    output_dir="./fine-tuned-model/swanglish_model/",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    weight_decay=1e-5,
    warmup_steps=50,
    num_train_epochs=50,# Adjust this based on your desired number of epochs
    save_total_limit=3,
    gradient_checkpointing=True,
    report_to=["tensorboard"], # Use TensorBoard for logging
    load_best_model_at_end=True, # Load the best model at the end of training
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    logging_dir="./fine-tuned-model/logs",
    generation_max_length=250,
    save_steps = 500,
    label_names=["labels"],
    greater_is_better=True
)

In [563]:
metric = evaluate.load("wer")

In [564]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

## Implementing Lora for more efficinet Fine Tuning of whisper

**LORA**- (Low-Rank Adaptation), is a training technique that reduces the number of trainable parameters.
It inserts a smaller number of new weights into the model and only these are trained.

This makes training with LoRA much faster, memory-efficient, and produces smaller model weights which are easier to store and share.

LoRA can also be combined with other training techniques like ' **DreamBooth**' to speedup the training process.

**Note** : Look into DreamBooth as well to see how training can be done faster


In [565]:
class ClearCache:
    def __enter__(self):
        torch.cuda.empty_cache()

    def __exit__(self, exc_type, exc_val, exc_tb):
        torch.cuda.empty_cache()

In [566]:
device

device(type='cuda')

In [567]:
def make_inputs_require_grad(module, input, output):
        output.requires_grad_(True)

In [568]:
PREFIX_CHECKPOINT_DIR = './fine-tuned-model/Peft-model/swanglish_model'


class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")

        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)

        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)
        return control


In [569]:
with ClearCache():

    model = prepare_model_for_kbit_training(model)

    model.model.encoder.conv1.register_forward_hook(make_inputs_require_grad)
    lora_config = LoraConfig(r=16, 
                             lora_alpha=32, 
                             target_modules=["q_proj", "v_proj"], 
                             lora_dropout=0.05, 
                             bias="none")

    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()

    trainer = Seq2SeqTrainer(
        args=training_args,
        model=model,
        train_dataset=split_dataset["train"],
        eval_dataset=split_dataset['test'],
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        tokenizer=processor.feature_extractor,
        callbacks=[SavePeftModelCallback]
    )

    
    
    trainer.train()

trainable params: 1,769,472 || all params: 243,504,384 || trainable%: 0.7266694631666262


/root/fine-tuning/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
500,1.696200,1.478978,135.403151


/root/fine-tuning/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/fine-tuning/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/fine-tuning/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differe

In [570]:
processor.save_pretrained(training_args.output_dir)

[]

In [571]:
trainer.model

PeftModel(
  (base_model): LoraModel(
    (model): WhisperForConditionalGeneration(
      (model): WhisperModel(
        (encoder): WhisperEncoder(
          (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
          (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
          (embed_positions): Embedding(1500, 768)
          (layers): ModuleList(
            (0-11): 12 x WhisperEncoderLayer(
              (self_attn): WhisperSdpaAttention(
                (k_proj): Linear(in_features=768, out_features=768, bias=False)
                (v_proj): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=768, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=16, bias=False)
                  )
                  (lora_B): ModuleDict

In [572]:
model.save_pretrained("./fine-tuned-small-sw/Normalsave/", from_pt=True)
# tokenizer.save_pretrained("./fine-tuned-tokenizer")
# feature_extractor.save_pretrained("./fine-tuned-feature-extractor")
# processor.save_pretrained("./fine-tuned-processor")

In [573]:
#model2 = whisper.from_pretrained('./whisper_model_saved_2')
from transformers import AutoModelForSpeechSeq2Seq,AutoModel,AutomaticSpeechRecognitionPipeline,TFPreTrainedModel

model2 = AutoModelForSpeechSeq2Seq.from_pretrained("/root/Notebooks/fine-tuned-small-sw/Normalsave", use_safetensors=True)
model2.to(device)

pre_tokenizer = AutoTokenizer.from_pretrained("/root/Notebooks/fine-tuned-tokenizer")
pre_feature_extractor = AutoFeatureExtractor.from_pretrained("/root/Notebooks/fine-tuned-feature-extractor/preprocessor_config.json")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [574]:
model2

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): lora.Linear(
              (base_layer): Linear(in_features=768, out_features=768, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.05, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=768, out_features=16, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=16, out_features=768, bias=False)
              )
              (lora_embed

In [575]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 221
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 44
    })
})

In [576]:
# import numpy as np
# # 
# # Define a function to convert list to numpy array
# def convert_list_to_ndarray(batch):
#     batch["input_ids"] = np.array(batch["input_ids"])
#     return batch

# # Apply the function to the dataset
# split_dataset = split_dataset.map(convert_list_to_ndarray)

In [577]:
transcriber_pipeline = pipeline("automatic-speech-recognition", model=model2, tokenizer=processor.tokenizer, feature_extractor = processor.feature_extractor)

In [578]:
or_split_dataset['test'][1]['audio']

{'path': None,
 'array': array([-0.00045776, -0.00039673, -0.00030518, ..., -0.04742432,
        -0.03793335, -0.03012085]),
 'sampling_rate': 16000}

In [584]:
audio_input = or_split_dataset['test'][4]['audio']
result = transcriber_pipeline(audio_input)

In [585]:
Sentence = or_split_dataset['test'][4]['sentence']

In [586]:
print('Generated Transcript = ',result)
print('Actual Transcript = ',Sentence)

Generated Transcript =  {'text': " it's cool this got really dark real fast real dark real real real fast okay and i think i should stop here before this gets emotional what i'm saying is i don't think it's right for you to put all that power with someone especially when they're just in high school because there are all these other kids who you don't want to be denying a chance that she's being"}
Actual Transcript =  at school this got really dark real fast real dark real real real fast okay and i think i.  should stop here before this gets emotional what i'm saying is i don't think it's right.  for you to put all that power with someone especially when they're just in high school.  because there are all these other kids who you don't want to be denying a chance that she's been


In [590]:
path = '/root/ccai/tmp/youtube_dataset/dataset_3/Swahili/24621936_rais-samia-asimulia-alivyofukuzwa-na-gari-la-polisi-akemea-lugha-za-kibaguzi_seg9.mp3'
result = transcriber_pipeline(path)

print(result)

{'text': ' ։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։։� ։'}


In [591]:
decoder = model2.model.decoder

In [ ]:
pre_feature_extractor = AutoFeatureExtractor.from_pretrained('/root/Notebooks/fine-tuned-feature-extractor/preprocessor_config.json')

In [ ]:
pipe = AutomaticSpeechRecognitionPipeline(
    model=model2,
    tokenizer=tokenizer,
    feature_extractor= pre_feature_extractor,
    #torch_dtype=model2.torch_dtype=,
    device=device,
    decoder=model2.model.decoder,
    #encoder=encoder
)

In [ ]:
all_predictions = []

# run streamed inference
for prediction in tqdm(
    pipe(
        KeyDataset(or_split_dataset['test'], "audio"),
        max_new_tokens=128,
        generate_kwargs={"task": "transcribe"},
        batch_size=32,
    ),
    total=len(or_split_dataset['test']),
):
    all_predictions.append(prediction["text"])

  0%|          | 0/7 [00:01<?, ?it/s]


ValueError: Multiple languages detected when trying to predict the most likely target language for transcription. It is currently not supported to transcribe to different languages in a single batch. Please make sure to either force a single language by passing `language='...'` or make sure all input audio is of the same language.

In [ ]:
class ClearCache:
    def __enter__(self):
        torch.cuda.empty_cache()

    def __exit__(self, exc_type, exc_val, exc_tb):
        torch.cuda.empty_cache()